In [0]:
import dlt
from pyspark.sql.functions import current_timestamp, col

@dlt.table(
    name="bronze.default.silver_orders", #USED Same Catalog due to ADB UI Issue, can use silver caatalog in ADB CLI
    comment="Silver table with non-null and unique order records"
)
@dlt.expect_or_fail("id_not_null", "id IS NOT NULL")
@dlt.expect_or_fail("category_not_null", "category IS NOT NULL")
@dlt.expect_or_fail("amount_not_null", "amount IS NOT NULL")
@dlt.expect_or_fail("unique_id", "id IS NOT NULL")
def silver_orders():
    return (
        dlt.read("bronze.default.orders")  # Bronze table
        .dropDuplicates(["id"])   # Enforces uniqueness
        .select(
            col("id").cast("int").alias("id"),
            col("category").cast("string").alias("category"),
            col("amount").cast("int").alias("amount"),
            col("ingestion_date").cast("timestamp").alias("ingestion_date")
        )
        .withColumn("silver_ingestion_date", current_timestamp())
    )
